In [108]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [109]:
from numpy import mean
from numpy import std
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_auc_score, mean_absolute_error
from sklearn import metrics

In [110]:
house_prices = pd.read_csv("../input/dataset-housingprices/dataset_housingprices.csv")
house_prices.shape #We find out there are 9 columns and 20640 rows

In [112]:
house_prices.head() #We find the different columns in the table

In [113]:
house_prices.isnull().sum() #We find that there are 207 entries in total_bedrooms column are NULL

In [114]:
house_prices.dropna(inplace=True)
house_prices.isnull().sum() #Removed null rows as 207 is pretty insignificant to 20640.

In [115]:
house_prices.describe()

In [132]:
house_prices.hist(bins=10, figsize=(15, 10)) 
#Plotting histograms, we find that median income is the only attribute that is normally distributed


In [117]:
fig, ax = plt.subplots()
ax.scatter(house_prices['longitude'], house_prices["median_house_value"])
 
#x-axis label
ax.set_xlabel('longitude')
 
# y-axis label
ax.set_ylabel('median_house_value')
plt.show()
#Scatter plot doesn't help determine

In [118]:
house_prices['total_rooms'] = np.log1p(house_prices['total_rooms'].values)
house_prices.total_rooms.hist()
#Applied log transforamtion on total_rooms to obtain normal distributions

In [119]:
house_prices['population'] = np.log1p(house_prices['population'].values)
house_prices.population.hist()
#Applied log transforamtion on population to obtain normal distributions

In [120]:
house_prices['total_bedrooms'] = np.log1p(house_prices['total_bedrooms'].values)
house_prices.total_bedrooms.hist()
#Applied log transforamtion on total_bedrooms to obtain normal distributions

In [121]:
house_prices['households'] = np.log1p(house_prices['households'].values)
house_prices.households.hist()
#Applied log transforamtion on households to obtain normal distributions

In [122]:
for column in house_prices:
    plt.figure()
    house_prices.boxplot([column])
    #Plotting boxplots helps us find tat the columns pouplation, total_rooms, total_bedrooms, household have outliers

In [123]:
for column in house_prices:
    
    data = house_prices[column]
    # calculate summary statistics
    data_mean, data_std = mean(data), std(data)
    # identify outliers
    cut_off = data_std * 3
    lower, upper = data_mean - cut_off, data_mean + cut_off
    # identify outliers
    outliers = [x for x in data if x < lower or x > upper]
    print(column)
    print('Identified outliers: %d' % len(outliers))
    # remove outliers
    outliers_removed = [x for x in data if x >= lower and x <= upper]
    print('Non-outlier observations: %d' % len(outliers_removed))
    house_prices = house_prices[(house_prices[column] < upper) & (house_prices[column] > lower)]
    
house_prices.shape

In [124]:
train_X = house_prices.loc[:, 'longitude':'median_income']
train_y = house_prices.loc[:, 'median_house_value']
train_X.head()

In [125]:
X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, test_size=0.3, random_state=1)#spilting train data

In [126]:
rf = LinearRegression() 

In [127]:
rf.fit(train_X, train_y) #Fitting model

In [128]:
X_val = X_val.loc[:, 'longitude':'median_income']
pred = rf.predict(X_val)

In [129]:
print(rf.score(X_val,y_val))

In [130]:
result=pd.DataFrame()
result['Expected'] = pd.DataFrame(pred)
result.head()

In [131]:
mae = mean_absolute_error(y_val, pred)
print(mae)